In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

file = r'E:\machin-prj\my_dataframe5.xlsx'
# 1. 데이터 로드 (my_dataframe5.xlsx)
df = pd.read_excel(file)


In [2]:
# # 1) 출발지 및 필터링
# start_station = '경복궁역'
# df_food = df[
#     (df['closest_subway_station'] == start_station) &
#     (df['cat1'] == '음식')
# ].copy()

# # 2) KMeans 클러스터링
# coords = df_food[['mapx','mapy']].values
# kmeans = KMeans(n_clusters=5, random_state=42).fit(coords)
# df_food['cluster'] = kmeans.labels_

# # 3) 각 군집의 대표 음식점 인덱스 수집
# rep_indices = []
# for i in range(5):
#     pts = df_food[df_food['cluster'] == i]
#     centroid = kmeans.cluster_centers_[i]
#     dists = np.linalg.norm(pts[['mapx','mapy']].values - centroid, axis=1)
#     rep_idx = pts.index[np.argmin(dists)]
#     rep_indices.append(rep_idx)

# # 4) 교통수단 라인 추출 함수
# def get_lines(row):
#     vals = row[['line1_x','line2_x','line3_x','line4_x','정류장번호']].dropna().astype(str)
#     return set(vals.values)

# # 5) 출발지 정보 로드
# start_row = df[df['closest_subway_station'] == start_station].iloc[0]
# current_coord = np.array([start_row.mapx, start_row.mapy])
# current_lines = get_lines(start_row)

# # 6) 그리디 순회 경로 생성
# remaining = rep_indices.copy()
# route = []
# while remaining:
#     candidates = [i for i in remaining if current_lines & get_lines(df_food.loc[i])]
#     if not candidates:
#         candidates = remaining.copy()
#     dists = [
#         np.linalg.norm(np.array([df_food.loc[i].mapx, df_food.loc[i].mapy]) - current_coord)
#         for i in candidates
#     ]
#     next_idx = candidates[int(np.argmin(dists))]
#     row = df_food.loc[next_idx]
#     shared = current_lines & get_lines(row)
#     transport = next(iter(shared)) if shared else next(iter(get_lines(row)))
#     route.append({'restaurant': row.title, 'transport': transport})
#     current_coord = np.array([row.mapx, row.mapy])
#     current_lines = get_lines(row)
#     remaining.remove(next_idx)

# # 7) 결과를 열로 저장
# cols = {}
# for i, step in enumerate(route, 1):
#     cols[f'음식점{i}']   = step['restaurant']
#     cols[f'이동수단{i}'] = step['transport']

# route_df = pd.DataFrame([cols])

In [3]:
# route_df

In [4]:
# df

In [5]:
# import pandas as pd
# from math import radians, sin, cos, sqrt, asin

# # Haversine 함수 정의
# def haversine(lat1, lon1, lat2, lon2):
#     lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
#     return 2 * 6371 * asin(sqrt(a))


# # 1) 음식 카테고리 필터링
# df_food = df[df['cat1'] == '음식'].copy()

# # 2) 경복궁역 좌표
# station_lat, station_lon = 37.579617, 126.977041

# # 3) 경복궁역에서 음식점까지 거리 계산
# df_food['distance'] = df_food.apply(
#     lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1
# )

# # 4) 가장 가까운 10개 음식점 추출
# top10 = df_food.nsmallest(10, 'distance').reset_index(drop=True)
# top10

In [6]:
# stage_names = top10["title"].tolist()  # ()를 붙여야 실제 리스트가 반환됩니다.
# print(stage_names)

In [7]:
# # 1) '음식' 카테고리 필터링
# df_food = df[df['cat1'] == '음식'].copy()

# # 2) 경복궁역 좌표
# station_lat, station_lon = 37.579617, 126.977041

# # 3) 경복궁역으로부터 거리 계산
# df_food['distance'] = df_food.apply(
#     lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1
# )

# # 4) 가장 가까운 10개 음식점 추출
# top10 = df_food.nsmallest(10, 'distance').reset_index(drop=True)

# # 5) stage_names 설정
# stage_names = top10["title"].tolist()

# # 추천 대상에서 top10 제외
# df_candidates = df_food.drop(top10.index)

# # 6) 각 스테이지별 2단계 추천 경로 생성 (5~10km 반경)
# routes = []
# for idx, row in top10.iterrows():
#     stage = stage_names[idx]
#     # 이동수단1, 출발/도착 설정
#     if idx % 2 == 0:
#         mode1, start1, end1 = '지하철', '경복궁역', row['closest_subway_station']
#     else:
#         mode1, start1, end1 = '버스', row['정류장번호'], row['matched_station_name']
    
#     # 1차 추천: 5~10km 내 5개
#     df_candidates['dist1'] = df_candidates.apply(
#         lambda r: haversine(row['mapy'], row['mapx'], r['mapy'], r['mapx']), axis=1
#     )
#     rec1_df = df_candidates[(df_candidates['dist1'] >= 5) & (df_candidates['dist1'] <= 10)]
#     rec1_df = rec1_df.nsmallest(5, 'dist1')
#     rec1_list = rec1_df['title'].tolist()
    
#     # 2차 추천: rec1_df 첫 번째 음식점 기준 5~10km 내 5개
#     if not rec1_df.empty:
#         rec1_first = rec1_df.iloc[0]
#         df_candidates2 = df_candidates.drop(rec1_df.index)
#         df_candidates2['dist2'] = df_candidates2.apply(
#             lambda r: haversine(rec1_first['mapy'], rec1_first['mapx'], r['mapy'], r['mapx']), axis=1
#         )
#         rec2_df = df_candidates2[(df_candidates2['dist2'] >= 5) & (df_candidates2['dist2'] <= 10)]
#         rec2_df = rec2_df.nsmallest(5, 'dist2')
#         rec2_list = rec2_df['title'].tolist()
#         # 이동수단2 설정 (mode1과 반대)
#         mode2 = '버스' if mode1 == '지하철' else '지하철'
#         if mode2 == '지하철':
#             start2 = rec1_first['closest_subway_station']
#             end2 = rec2_df.iloc[0]['closest_subway_station'] if not rec2_df.empty else None
#         else:
#             start2 = rec1_first['정류장번호']
#             end2 = rec2_df.iloc[0]['matched_station_name'] if not rec2_df.empty else None
#     else:
#         mode2, start2, end2, rec2_list = None, None, None, []
    
#     routes.append({
#         '단계': stage,
#         '이동수단1': mode1,
#         '타는곳1': start1,
#         '내리는곳1': end1,
#         '추천음식점1': ', '.join(rec1_list),
#         '이동수단2': mode2,
#         '타는곳2': start2,
#         '내리는곳2': end2,
#         '추천음식점2': ', '.join(rec2_list)
#     })

# # 결과 DataFrame 표시
# result_df = pd.DataFrame(routes)
# result_df

In [8]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from math import radians

In [9]:
# df_food = df[df['cat1'] == '음식'].reset_index(drop=True)

# # BallTree 생성 (haversine 거리)
# coords_rad = np.radians(df_food[['mapy', 'mapx']].values)
# tree = BallTree(coords_rad, metric='haversine')

# # 추천 함수: low_km <= distance <= high_km, 최대 k개
# def recommend_km_range(lat, lon, low_km, high_km, k):
#     point = np.radians([[lat, lon]])
#     idxs = tree.query_radius(point, r=high_km/6371)[0]
#     if len(idxs) == 0:
#         return pd.DataFrame(columns=df_food.columns)
#     dists = tree.query(point, k=len(idxs))[0][0] * 6371
#     valid_ids = idxs[(dists >= low_km)]
#     valid_ids = valid_ids[:k]
#     return df_food.iloc[valid_ids]

# # 초기 스테이지: 경복궁역 기준 0~10km에서 10개
# station_lat, station_lon = 37.579617, 126.977041
# initial = recommend_km_range(station_lat, station_lon, 0, 10, 10)

# # 2단계 ML 경로 추천 (boarding != alighting 보장)
# routes = []
# for idx, row in initial.iterrows():
#     stage = row['title']
#     # 이동수단1 결정
#     if idx % 2 == 0:
#         mode1 = '지하철'
#         start1 = '경복궁역'
#         end1 = row['closest_subway_station']
#     else:
#         mode1 = '버스'
#         start1 = row['정류장번호']
#         end1 = row['matched_station_name']
#     # 1차 추천 (5~10km, up to 5개)
#     rec1_df = recommend_km_range(row['mapy'], row['mapx'], 5, 10, 5)
#     rec1_list = rec1_df['title'].tolist()
#     # 이동수단2 (반대)
#     mode2 = '버스' if mode1 == '지하철' else '지하철'
#     # 2차 추천: 첫 추천 지점 기준
#     if not rec1_df.empty:
#         first = rec1_df.iloc[0]
#         # find rec2 candidates
#         rec2_df = recommend_km_range(first['mapy'], first['mapx'], 5, 10, 10)
#         # filter out same boarding and alighting
#         if mode2 == '지하철':
#             rec2_df = rec2_df[rec2_df['closest_subway_station'] != first['closest_subway_station']]
#             start2 = first['closest_subway_station']
#             end2 = rec2_df.iloc[0]['closest_subway_station'] if not rec2_df.empty else None
#         else:
#             rec2_df = rec2_df[rec2_df['matched_station_name'] != first['matched_station_name']]
#             start2 = first['matched_station_name']
#             end2 = rec2_df.iloc[0]['matched_station_name'] if not rec2_df.empty else None
#         rec2_list = rec2_df['title'].tolist()[:5]
#     else:
#         start2 = end2 = mode2 = None
#         rec2_list = []
#     routes.append({
#         '단계': stage,
#         '이동수단1': mode1,
#         '타는곳1': start1,
#         '내리는곳1': end1,
#         '추천음식점1': ', '.join(rec1_list),
#         '이동수단2': mode2,
#         '타는곳2': start2,
#         '내리는곳2': end2,
#         '추천음식점2': ', '.join(rec2_list)
#     })

# # 결과 출력
# result_df = pd.DataFrame(routes)

In [10]:
# result_df

In [11]:
file_review = r'E:\machin-prj\review_data_인천_행정동추가.csv'
df_review = pd.read_csv(file_review, encoding='utf-8')
df_review.head(2)

,검색어,장소명,평점,주소,리뷰닉네임,별점,날짜,리뷰내용,title,addr1,contentid,contenttypeid,유형,연도월,new_cat,행정동명,자치구,경도,위도
0,감중공원,감중공원,3.3,인천광역시 서구 가정로98번길 18,헷,5.0,2024.11.14.,NaN,감중공원,인천광역시 서구 가정로98번길 18,2767573.0,12.0,관광지,2024-11,힐링,가좌동,서구,126.662197,37.491604
1,갑곶돈대,갑곶돈대,4.4,인천광역시 강화군 강화읍 갑곳리 1022,병권💚,5.0,2025.04.06.,시설이 좋아요 😀 ㅎㅎ,갑곶돈대,인천광역시 강화군 강화읍 해안동로1366번길 18,1254680.0,12.0,관광지,2025-04,문화,강화읍,강화군,126.488237,37.751891


sent_text: 텍스트만으로 분석한 감정

sent_score: 별점만으로 분류한 감정 (45점 긍정, 12점 부정, 3점 중립)

sentiment: 텍스트가 없을 때 sent_score, 텍스트가 있을 때 sent_text를 사용

In [12]:
# import pandas as pd

# # 1️⃣ 리뷰 점수 평균 계산 (장소명 기준)
# df_avg_score = df_review.groupby('장소명', as_index=False)['별점'].mean()
# df_avg_score.rename(columns={'별점': 'avg_rating_score'}, inplace=True)

# # 2️⃣ 텍스트 기반 감정 점수 생성
# pos_words = ["좋다", "최고", "만족", "추천", "행복", "감사", "친절", "편리", "맛있"]
# neg_words = ["나쁘다", "불만", "최악", "짜증", "불편", "실망", "불친절", "별로"]

# def get_text_sentiment(text):
#     text = str(text)
#     pos = sum(text.count(w) for w in pos_words)
#     neg = sum(text.count(w) for w in neg_words)
#     if pos > neg:
#         return 10
#     elif neg > pos:
#         return 5
#     else:
#         return 0

# df_review['sent_text_score'] = df_review['리뷰내용'].apply(get_text_sentiment)

# # 3️⃣ 텍스트 감정 점수 평균 계산 (장소명 기준)
# df_avg_text = df_review.groupby('장소명', as_index=False)['sent_text_score'].mean()
# df_avg_text.rename(columns={'sent_text_score': 'avg_text_sent_score'}, inplace=True)

# # 4️⃣ 리뷰 점수와 감정 점수 결합
# df_final = pd.merge(df_avg_score, df_avg_text, on='장소명', how='left')

# # 5️⃣ 최종 점수 계산 (두 점수 평균)
# df_final['final_score'] = (df_final['avg_rating_score'] + df_final['avg_text_sent_score']) / 2

# # 6️⃣ 결과 출력
# df_final.head()


In [13]:
import pandas as pd

# 1️⃣ 리뷰 점수 평균 계산
df_avg_score = df_review.groupby('장소명', as_index=False)['별점'].mean()
df_avg_score.rename(columns={'별점': 'avg_rating_score'}, inplace=True)

# 2️⃣ 텍스트 기반 감정 점수 생성
pos_words = ["좋다", "최고", "만족", "추천", "행복", "감사", "친절", "편리", "맛있"]
neg_words = ["나쁘다", "불만", "최악", "짜증", "불편", "실망", "불친절", "별로"]

def get_text_sentiment(text):
    text = str(text)
    pos = sum(text.count(w) for w in pos_words)
    neg = sum(text.count(w) for w in neg_words)
    if pos > neg:
        return 10
    elif neg > pos:
        return 5
    else:
        return 0

df_review['sent_text_score'] = df_review['리뷰내용'].apply(get_text_sentiment)

# 3️⃣ 텍스트 감정 점수 평균 계산
df_avg_text = df_review.groupby('장소명', as_index=False)['sent_text_score'].mean()
df_avg_text.rename(columns={'sent_text_score': 'avg_text_sent_score'}, inplace=True)

# 4️⃣ 원본 데이터 유지하며 리뷰 점수 및 감정 점수 추가
df_final = df_review.drop_duplicates(subset=['장소명']).copy()  # 장소별 중복 제거
df_final = pd.merge(df_final, df_avg_score, on='장소명', how='left')  # 리뷰 점수 추가
df_final = pd.merge(df_final, df_avg_text, on='장소명', how='left')  # 감정 점수 추가

# 5️⃣ 최종 점수 계산 (별점 평균 + 감정 점수 평균) / 2
df_final['final_score'] = (df_final['avg_rating_score'] + df_final['avg_text_sent_score']) / 2

# 6️⃣ 결과 출력
df_final.head(2)


,검색어,장소명,평점,주소,리뷰닉네임,별점,날짜,리뷰내용,title,addr1,...,연도월,new_cat,행정동명,자치구,경도,위도,sent_text_score,avg_rating_score,avg_text_sent_score,final_score
0,감중공원,감중공원,3.3,인천광역시 서구 가정로98번길 18,헷,5.0,2024.11.14.,NaN,감중공원,인천광역시 서구 가정로98번길 18,...,2024-11,힐링,가좌동,서구,126.662197,37.491604,0,5.000000,0.0,2.500000
1,갑곶돈대,갑곶돈대,4.4,인천광역시 강화군 강화읍 갑곳리 1022,병권💚,5.0,2025.04.06.,시설이 좋아요 😀 ㅎㅎ,갑곶돈대,인천광역시 강화군 강화읍 해안동로1366번길 18,...,2025-04,문화,강화읍,강화군,126.488237,37.751891,0,4.222222,0.0,2.111111


In [14]:
# import pandas as pd
# import numpy as np
# from sklearn.cluster import KMeans

# file = r'E:\machin-prj\my_dataframe5.xlsx'
# # 1. 데이터 로드 (my_dataframe5.xlsx)
# df = pd.read_excel(file)


In [15]:

df = pd.merge(df, df_final[['addr1', 'final_score']], how='left', on='addr1')
df = df.drop_duplicates(subset=['title'], keep='first').reset_index(drop=True)
df.head(2)

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,final_score
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트(중),GGB123000013,08호선,NaN,NaN,NaN,푼주,NaN
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역,상왕십리역,상왕십리역,GGB103000005,02호선,NaN,NaN,NaN,레이지 유스,NaN


In [16]:
import pandas as pd
from math import radians, sin, cos, sqrt, asin

# Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

# 1) 음식 카테고리 필터링
df_food = df[df['cat1'] == '음식'].copy()

# 2) 인천역 좌표
station_lat, station_lon = 37.476919, 126.616981

# 3) 각 음식점까지 거리 계산
df_food['distance'] = df_food.apply(
    lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1
)

# 4) 가장 가까운 10개 음식점 추출
top10_closest = df_food.nsmallest(10, 'distance').reset_index(drop=True)

# 5) final_score 기준으로 정렬
top10_sorted = top10_closest.sort_values(by='final_score', ascending=False).reset_index(drop=True)


top10_sorted

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,final_score,distance
0,인천광역시 중구 차이나타운로44번길 31-3,음식,음식점,한식,126.617762,37.475556,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,신승반점,4.687500,0.166531
1,인천광역시 중구 차이나타운로 36 만다복,음식,음식점,한식,126.619287,37.476175,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,만다복,4.400000,0.219654
2,인천광역시 중구 차이나타운로 50-2,음식,음식점,중식,126.618795,37.475145,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,십리향,4.052632,0.254091
3,인천광역시 중구 차이나타운로 41-1,음식,음식점,중식,126.619729,37.475492,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,청관,3.562500,0.289775
4,인천광역시 중구 차이나타운로 43,음식,음식점,중식,126.619590,37.475458,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,공화춘,2.287500,0.281722
5,인천광역시 중구 차이나타운로 20,음식,음식점,중식,126.619084,37.477553,송월동동화마을,인천역,송월동동화마을,ICB161000189,01호선,수인선,NaN,NaN,문차이나,2.153846,0.198508
6,인천광역시 중구 차이나타운로52번길 14,음식,음식점,한식,126.618059,37.475211,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,서산밴댕이,1.500000,0.212399
7,인천광역시 중구 차이나타운로44번길 16-24,음식,음식점,카페/전통찻집,126.618842,37.476317,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,아키라커피 본점,NaN,0.177393
8,인천광역시 중구 차이나타운로 56 (선린동),음식,음식점,중식,126.618479,37.474775,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,풍미,NaN,0.272567
9,인천광역시 중구 차이나타운로 53,음식,음식점,중식,126.618795,37.474820,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,중국성,NaN,0.283067


In [17]:
# # 4) 가장 가까운 10개 음식점 추출
# top10 = df_food.nsmallest(10, 'distance').reset_index(drop=True)
# top10

In [18]:
# selected_titles = set()  # 중복 방지를 위한 집합

# def get_unique_recommendations(df, column, num):
#     global selected_titles
#     recommendations = []
    
#     for _, row in df.nsmallest(num * 2, column).iterrows():  # 후보군을 넉넉히 가져오기
#         if row['title'] not in selected_titles:  # 중복 체크
#             recommendations.append(row)
#             selected_titles.add(row['title'])  # 선택된 음식점 저장
#         if len(recommendations) == num:
#             break
    
#     return pd.DataFrame(recommendations)

# top10_positive = get_unique_recommendations(df_food[df_food['sent_score'] == 'positive'], 'distance', 50)

In [19]:
# top10_positive

In [20]:

stage_names = top10_sorted["title"].tolist()

In [21]:
stage_names

['신승반점', '만다복', '십리향', '청관', '공화춘', '문차이나', '서산밴댕이', '아키라커피 본점', '풍미', '중국성']

In [22]:
# import pandas as pd
# from math import radians, sin, cos, sqrt, asin

# # Haversine 거리 계산 함수
# def haversine(lat1, lon1, lat2, lon2):
#     lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
#     dlat, dlon = lat2 - lat1, lon2 - lon1
#     a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
#     return 2 * 6371 * asin(sqrt(a))

# # 1️⃣ '음식' 카테고리 필터링
# df_food = df[df['cat1'] == '음식'].copy()

# # 2️⃣ 인천역 좌표
# station_lat, station_lon = 37.476919, 126.616981

# # 3️⃣ 각 음식점까지 거리 계산
# df_food['distance'] = df_food.apply(lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1)

# # 4️⃣ 가장 가까운 10개 음식점 추출 후, 별점 높은 순으로 정렬
# top10_sorted = df_food.nsmallest(10, 'distance').sort_values(by='final_score', ascending=False).reset_index(drop=True)

# # 5️⃣ 추천 대상에서 top10 제외
# df_candidates = df_food.drop(top10_sorted.index)

# # 6️⃣ 각 스테이지별 2단계 추천 경로 생성 (5~10km 반경)
# routes = []
# for idx, row in top10_sorted.iterrows():
#     stage = row['title']
    
#     # 이동수단 설정
#     mode1 = '지하철' if idx % 2 == 0 else '버스'
#     start1 = '인천역' if mode1 == '지하철' else row['정류장번호']
#     end1 = row['closest_subway_station'] if mode1 == '지하철' else row['matched_station_name']
    
#     # 1차 추천: 거리 내 음식점 5개, **별점 높은 순 정렬 추가**
#     df_candidates['dist1'] = df_candidates.apply(lambda r: haversine(row['mapy'], row['mapx'], r['mapy'], r['mapx']), axis=1)
#     rec1_df = df_candidates[(df_candidates['dist1'] >= 5) & (df_candidates['dist1'] <= 10)]
#     rec1_df = rec1_df.nsmallest(10, 'dist1').sort_values(by='final_score', ascending=False).head(5)
#     rec1_list = rec1_df['title'].tolist()
    
#     # 2차 추천: rec1_df 첫 번째 음식점 기준 5~10km 내 5개 (별점 높은 순 정렬 추가)
#     if not rec1_df.empty:
#         rec1_first = rec1_df.iloc[0]
#         df_candidates2 = df_candidates.drop(rec1_df.index)
#         df_candidates2['dist2'] = df_candidates2.apply(lambda r: haversine(rec1_first['mapy'], rec1_first['mapx'], r['mapy'], r['mapx']), axis=1)
#         rec2_df = df_candidates2[(df_candidates2['dist2'] >= 5) & (df_candidates2['dist2'] <= 10)]
#         rec2_df = rec2_df.nsmallest(10, 'dist2').sort_values(by='final_score', ascending=False).head(5)
#         rec2_list = rec2_df['title'].tolist()
        
#         # 이동수단2 설정
#         mode2 = '버스' if mode1 == '지하철' else '지하철'
#         start2 = rec1_first['closest_subway_station'] if mode2 == '지하철' else rec1_first['정류장번호']
#         end2 = rec2_df.iloc[0]['closest_subway_station'] if mode2 == '지하철' and not rec2_df.empty else rec2_df.iloc[0]['matched_station_name'] if not rec2_df.empty else None
#     else:
#         mode2, start2, end2, rec2_list = None, None, None, []

#     # 추천 경로 저장
#     routes.append({
#         '단계': stage,
#         '이동수단1': mode1,
#         '타는곳1': start1,
#         '내리는곳1': end1,
#         '추천음식점1': ', '.join(rec1_list),
#         '이동수단2': mode2,
#         '타는곳2': start2,
#         '내리는곳2': end2,
#         '추천음식점2': ', '.join(rec2_list)
#     })

# # 7️⃣ 결과 DataFrame 생성 및 출력
# result_df = pd.DataFrame(routes)
# result_df


In [23]:
# import pandas as pd
# import numpy as np

# # # 1) '음식' 카테고리 + sentiment가 'positive'인 데이터만 필터링
# # df_food = df[(df['cat1'] == '음식') & (df['sent_score'] == 'positive')].reset_index(drop=True)



# # 추천 대상에서 top10 제외
# df_candidates = df_food.drop(top10_sorted.index)

# # 6) 각 스테이지별 2단계 추천 경로 생성 (5~10km 반경)
# routes = []
# for idx, row in top10_sorted.iterrows():
#     stage = stage_names[idx]
#     # 이동수단1, 출발/도착 설정
#     if idx % 2 == 0:
#         mode1, start1, end1 = '지하철', '인천역', row['closest_subway_station']
#     else:
#         mode1, start1, end1 = '버스', row['정류장번호'], row['matched_station_name']
    
#     # 1차 추천: 5~10km 내 5개
#     df_candidates['dist1'] = df_candidates.apply(
#         lambda r: haversine(row['mapy'], row['mapx'], r['mapy'], r['mapx']), axis=1
#     )
#     rec1_df = df_candidates[(df_candidates['dist1'] >= 5) & (df_candidates['dist1'] <= 10)]
#     rec1_df = rec1_df.nsmallest(5, 'dist1')
#     rec1_list = rec1_df['title'].tolist()
    
#     # 2차 추천: rec1_df 첫 번째 음식점 기준 5~10km 내 5개
#     if not rec1_df.empty:
#         rec1_first = rec1_df.iloc[0]
#         df_candidates2 = df_candidates.drop(rec1_df.index)
#         df_candidates2['dist2'] = df_candidates2.apply(
#             lambda r: haversine(rec1_first['mapy'], rec1_first['mapx'], r['mapy'], r['mapx']), axis=1
#         )
#         rec2_df = df_candidates2[(df_candidates2['dist2'] >= 5) & (df_candidates2['dist2'] <= 10)]
#         rec2_df = rec2_df.nsmallest(5, 'dist2')
#         rec2_list = rec2_df['title'].tolist()
#         # 이동수단2 설정 (mode1과 반대)
#         mode2 = '버스' if mode1 == '지하철' else '지하철'
#         if mode2 == '지하철':
#             start2 = rec1_first['closest_subway_station']
#             end2 = rec2_df.iloc[0]['closest_subway_station'] if not rec2_df.empty else None
#         else:
#             start2 = rec1_first['정류장번호']
#             end2 = rec2_df.iloc[0]['matched_station_name'] if not rec2_df.empty else None
#     else:
#         mode2, start2, end2, rec2_list = None, None, None, []
    
#     routes.append({
#         '단계': stage,
#         '이동수단1': mode1,
#         '타는곳1': start1,
#         '내리는곳1': end1,
#         '추천음식점1': ', '.join(rec1_list),
#         '이동수단2': mode2,
#         '타는곳2': start2,
#         '내리는곳2': end2,
#         '추천음식점2': ', '.join(rec2_list)
#     })

# # 결과 DataFrame 표시
# result_df = pd.DataFrame(routes)
# result_df

In [24]:
# import pandas as pd
# from math import radians, sin, cos, sqrt, asin

# # Haversine 거리 계산 함수
# def haversine(lat1, lon1, lat2, lon2):
#     lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
#     dlat, dlon = lat2 - lat1, lon2 - lon1
#     a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
#     return 2 * 6371 * asin(sqrt(a))

# # 1️⃣ 음식 카테고리 필터링 및 추천 음식점 제외
# df_food = df[df['cat1'] == '음식'].copy()

# # 2️⃣ 인천역 좌표
# station_lat, station_lon = 37.476919, 126.616981

# # 3️⃣ 각 음식점까지 거리 계산
# df_food['distance'] = df_food.apply(lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1)

# # 4️⃣ 가장 가까운 음식점 10개 선택 후 final_score 높은 순으로 정렬
# top10_sorted = df_food.nsmallest(10, 'distance').sort_values(by='final_score', ascending=False).reset_index(drop=True)

# # 5️⃣ 추천 대상에서 이미 선택된 음식점 제외
# selected_titles = set(top10_sorted['title'])
# df_candidates = df_food[~df_food['title'].isin(selected_titles)].copy()

# # 6️⃣ 각 스테이지별 2단계 추천 (5~10km 반경, 별점 높은 순)
# routes = []
# for idx, row in top10_sorted.iterrows():
#     stage = row['title']
    
#     # 이동수단 설정
#     mode1 = '지하철' if idx % 2 == 0 else '버스'
#     start1 = '인천역' if mode1 == '지하철' else row['정류장번호']
#     end1 = row['closest_subway_station'] if mode1 == '지하철' else row['matched_station_name']

#     # 1차 추천: 가까운 음식점 5개, 중복 제외 및 별점 높은 순 정렬
#     df_candidates['dist1'] = df_candidates.apply(lambda r: haversine(row['mapy'], row['mapx'], r['mapy'], r['mapx']), axis=1)
#     rec1_df = df_candidates[(df_candidates['dist1'] >= 5) & (df_candidates['dist1'] <= 10)]
#     rec1_df = rec1_df.nsmallest(10, 'dist1').sort_values(by='final_score', ascending=False).head(5)

#     # 🔥 중복 음식점 제거 🔥
#     rec1_list = [t for t in rec1_df['title'].tolist() if t not in selected_titles]
#     selected_titles.update(rec1_list)  # 추천 음식점 추가

#     # 2차 추천: 첫 추천 음식점 기준으로 5~10km 내에서 별점 높은 순 정렬
#     if rec1_list:
#         rec1_first = rec1_df.iloc[0]
#         df_candidates2 = df_candidates.drop(rec1_df.index)
#         df_candidates2['dist2'] = df_candidates2.apply(lambda r: haversine(rec1_first['mapy'], rec1_first['mapx'], r['mapy'], r['mapx']), axis=1)
#         rec2_df = df_candidates2[(df_candidates2['dist2'] >= 5) & (df_candidates2['dist2'] <= 10)]
#         rec2_df = rec2_df.nsmallest(10, 'dist2').sort_values(by='final_score', ascending=False).head(5)

#         # 🔥 중복 음식점 제거 🔥
#         rec2_list = [t for t in rec2_df['title'].tolist() if t not in selected_titles]
#         selected_titles.update(rec2_list)  # 추천 음식점 추가
        
#         # 이동수단2 설정
#         mode2 = '버스' if mode1 == '지하철' else '지하철'
#         start2 = rec1_first['closest_subway_station'] if mode2 == '지하철' else rec1_first['정류장번호']
#         end2 = rec2_df.iloc[0]['closest_subway_station'] if mode2 == '지하철' and rec2_list else rec2_df.iloc[0]['matched_station_name'] if rec2_list else None
#     else:
#         mode2, start2, end2, rec2_list = None, None, None, []

#     # 추천 경로 저장
#     routes.append({
#         '단계': stage,
#         '이동수단1': mode1,
#         '타는곳1': start1,
#         '내리는곳1': end1,
#         '추천음식점1': ', '.join(rec1_list),
#         '이동수단2': mode2,
#         '타는곳2': start2,
#         '내리는곳2': end2,
#         '추천음식점2': ', '.join(rec2_list)
#     })

# # 7️⃣ 결과 DataFrame 생성 및 출력
# result_df = pd.DataFrame(routes)
# result_df

In [25]:
from sklearn.neighbors import NearestNeighbors
# # 1) '음식' 카테고리 + sentiment가 'positive'인 데이터만 필터링
# # df_food = df[(df['cat1']=='음식') & (df['sent_score']=='positive')].reset_index(drop=True)

# # 2) 추천 대상에서 top10 제외
# df_candidates = df_food.drop(top10_sorted.index)

# # 3) 좌표를 라디안(radian)으로 변환 (haversine metric 사용을 위해)
# coords = np.radians(df_candidates[['mapy','mapx']].values)

# # 지구 반지름 (km)
# EARTH_RADIUS = 6371.0

# # 4) NearestNeighbors 모델 생성 (반경 10km까지, haversine 거리 측정)
# nn = NearestNeighbors(
#     n_neighbors=30,
#     radius=10 / EARTH_RADIUS,
#     metric='haversine'
# )
# nn.fit(coords)

# routes = []
# for idx, row in top10_sorted.iterrows():
#     stage = stage_names[idx]
#     # 출발 지점 좌표 (radian)
#     pt = np.radians([[row['mapy'], row['mapx']]])
    
#     # 1차 추천
#     dists, idxs = nn.kneighbors(pt, n_neighbors=30)
#     dists_km = dists[0] * EARTH_RADIUS
#     # 5km 이상 10km 이하 필터
#     mask1 = (dists_km >= 5) & (dists_km <= 10)
#     rec1_idxs = idxs[0][mask1][:5]
#     rec1_df = df_candidates.iloc[rec1_idxs]
#     rec1_list = rec1_df['title'].tolist()
    
#     # 2차 추천 (첫 번째 추천을 기준으로 다시 검색)
#     if len(rec1_idxs) > 0:
#         first_pt = np.radians([rec1_df.iloc[0][['mapy','mapx']].values])
#         d2, i2 = nn.kneighbors(first_pt, n_neighbors=30)
#         d2_km = d2[0] * EARTH_RADIUS
#         mask2 = (d2_km >= 5) & (d2_km <= 10)
#         rec2_idxs = i2[0][mask2][:5]
#         # 중복 제거
#         rec2_idxs = [i for i in rec2_idxs if i not in rec1_idxs]
#         rec2_df = df_candidates.iloc[rec2_idxs]
#         rec2_list = rec2_df['title'].tolist()
#     else:
#         rec2_list = []
    
#     # 이동수단과 역/정류장 정보는 기존 로직 그대로 사용
#     if idx % 2 == 0:
#         mode1, start1, end1 = '지하철', '인천역', row['closest_subway_station']
#         mode2 = '버스'
#         start2 = rec1_df.iloc[0]['closest_subway_station'] if not rec1_df.empty else None
#         end2 = rec2_df.iloc[0]['matched_station_name'] if rec2_list else None
#     else:
#         mode1, start1, end1 = '버스', row['정류장번호'], row['matched_station_name']
#         mode2 = '지하철'
#         start2 = rec1_df.iloc[0]['정류장번호'] if not rec1_df.empty else None
#         end2 = rec2_df.iloc[0]['closest_subway_station'] if rec2_list else None

#     routes.append({
#         '단계': stage,
#         '이동수단1': mode1,
#         '타는곳1': start1,
#         '내리는곳1': end1,
#         '추천음식점1': ', '.join(rec1_list),
#         '이동수단2': mode2,
#         '타는곳2': start2,
#         '내리는곳2': end2,
#         '추천음식점2': ', '.join(rec2_list)
#     })

# # 결과 DataFrame 생성
# result_df = pd.DataFrame(routes)
# result_df

In [26]:
df_candidates = df_food.drop(top10_sorted.index)
coords_rad = np.radians(df_candidates[['mapy','mapx']].values)
EARTH_RADIUS = 6371.0

# 4) 반경 10km NN 모델 (haversine)
nn = NearestNeighbors(radius=10/EARTH_RADIUS, metric='haversine')
nn.fit(coords_rad)

used_idxs = set()
routes = []

for idx, row in top10_sorted.iterrows():
    stage = stage_names[idx]
    pt1_rad = np.radians([[row['mapy'], row['mapx']]])

    # — 1차 추천: 5~10km 필터 후 score↓, dist↑ —
    d1_rad, i1 = nn.radius_neighbors(pt1_rad, return_distance=True)
    d1_km = d1_rad[0] * EARTH_RADIUS

    # 5km 이상만
    mask1 = d1_km >= 5
    cand1_all = i1[0][mask1]
    dist1_all = d1_km[mask1]

    # 사용 안 한 것만 모아서 DataFrame
    pairs1 = [(i, dist) for i, dist in zip(cand1_all, dist1_all) if i not in used_idxs]
    tmp1 = pd.DataFrame(pairs1, columns=['idx','dist'])
    # iloc으로 final_score 불러오기
    tmp1['final_score'] = df_candidates.iloc[tmp1['idx']]['final_score'].values
    # score 내림차순, 거리 오름차순
    tmp1 = tmp1.sort_values(['final_score','dist'], ascending=[False,True])

    rec1_idxs = tmp1['idx'].tolist()[:5]
    used_idxs.update(rec1_idxs)

    rec1_df   = df_candidates.iloc[rec1_idxs]
    rec1_list = rec1_df['title'].tolist()
    rec1_first= rec1_df.iloc[0] if not rec1_df.empty else None

    # — 2차 추천 (rec1_first 기준 동일 방식) —
    rec2_idxs = []
    if rec1_first is not None:
        pt2_rad = np.radians([[rec1_first['mapy'], rec1_first['mapx']]])
        d2_rad, i2 = nn.radius_neighbors(pt2_rad, return_distance=True)
        d2_km = d2_rad[0] * EARTH_RADIUS

        mask2 = d2_km >= 5
        cand2_all = i2[0][mask2]
        dist2_all = d2_km[mask2]

        pairs2 = [(i, dist) for i, dist in zip(cand2_all, dist2_all) if i not in used_idxs]
        tmp2 = pd.DataFrame(pairs2, columns=['idx','dist'])
        tmp2['final_score'] = df_candidates.iloc[tmp2['idx']]['final_score'].values
        tmp2 = tmp2.sort_values(['final_score','dist'], ascending=[False,True])

        rec2_idxs = tmp2['idx'].tolist()[:5]
        used_idxs.update(rec2_idxs)

    rec2_df    = df_candidates.iloc[rec2_idxs]
    rec2_list  = rec2_df['title'].tolist()
    rec2_first = rec2_df.iloc[0] if not rec2_df.empty else None

    # — 이동수단1 / 내리는곳1 결정 —
    if idx % 2 == 0:
        mode1, start1 = '지하철', '인천역'
    else:
        mode1, start1 = '버스', row['정류장번호']

    if rec1_first is not None:
        end1 = (rec1_first['closest_subway_station']
                if mode1=='지하철'
                else rec1_first['matched_station_name'])
    else:
        end1 = None

    # — 이동수단2 / 타는곳2, 내리는곳2 결정 —
    mode2 = '버스' if mode1=='지하철' else '지하철'
    if rec1_first is not None:
        start2 = (rec1_first['closest_subway_station']
                  if mode2=='지하철'
                  else rec1_first['matched_station_name'])
    else:
        start2 = None
    if rec2_first is not None:
        end2 = (rec2_first['closest_subway_station']
                if mode2=='지하철'
                else rec2_first['matched_station_name'])
    else:
        end2 = None

    routes.append({
        '단계': stage,
        '이동수단1': mode1,
        '타는곳1': start1,
        '내리는곳1': end1,
        '추천음식점1': ', '.join(rec1_list) or '없음',
        '이동수단2': mode2,
        '타는곳2': start2,
        '내리는곳2': end2,
        '추천음식점2': ', '.join(rec2_list) or '없음'
    })

result_df = pd.DataFrame(routes)
result_df

,단계,이동수단1,타는곳1,내리는곳1,추천음식점1,이동수단2,타는곳2,내리는곳2,추천음식점2
0,신승반점,지하철,인천역,서구청역,"한울능이버섯백숙, 소담잔치국수, 강선생막국수, 도톰카츠 청라점, 서당골막국수",버스,웰카운티1차,용일초등학교,"연안별버거, 멜란지베이커리, 퍼플스왈로우, 계산동수제돈까스, [백년가게]부영선지국"
1,만다복,버스,GGB162000136,센트럴파크푸르지오아파트,"송쭈집 본점, 청년쌈밥, 솔청청, 청라가야밀면국밥, 잊지못해꽃게탕",지하철,센트럴파크역,인천논현역,"멜팅룸, 익선우대갈비, 연안부두 목포신안18호횟집, 문화반점, 화양계곡민물매운탕"
2,십리향,지하철,인천역,센트럴파크역,"따룽훠궈 송도점, 뉴욕베이글 송도, 올드타운, 3일, 송도갈매기",버스,센트럴파크푸르지오아파트,롯데캐슬모래내시장,"병천찹쌀순대, 전복촌, 얼레꼴레만두, 불티나왕돈까스 본점, 신승반점"
3,청관,버스,GGB162000136,가재울사거리,"신진말, 미나모토, 서해수산, 시즈쿠, 전구식당",지하철,인천가좌역,만수역,"미소로, 청라써니네집, 은옥, 제이콥스 핏제리아, 군봉묵은지김치찜"
4,공화춘,지하철,인천역,영종역,"조은전골칼국수, 미담 맛을 논하다, 온누리쭈꾸미부평본점, 영종원조국수, 셀레브르",버스,영종교회,장안사거리,"길손물텀벙, 비스트로페르레이, 만다복, 카페시나브로, 동락반점"
5,문차이나,버스,ICB161000189,센트럴파크1,"엉클인더키친 센트럴파크 본점, 임파스토, 호가화, 송도 수블라키아, 장독집 청량산단상",지하철,센트럴파크역,석바위시장역,"연중반점, 유노미, 갯마을셀프회, 월미도회집, 이화찹쌀순대"
6,서산밴댕이,지하철,인천역,간석오거리역,"해중루, 머노까머나 송도점, 아라짬뽕, 옥골한우정육식당, 숙성명작",버스,간석오거리,삼천리3차아파트,"통큰손쭈꾸미마을계산본점, 가든파티 계양점, 노란우체통, 청라우사미, 금강산흑염소전문점"
7,아키라커피 본점,버스,GGB162000136,가재울사거리,"파운드 바베큐, 빨간돼지 청라점, 한우야, 석남동함흥냉면, 해미루",지하철,인천가좌역,갈산역,"보릿골 정찬, 장수가든, 십리향, 크로마이트커피, 휴그릴"
8,풍미,지하철,인천역,센트럴파크역,"스시이와, ZOUMA, 오구당당 부평본점, 송도주막, 만정수산",버스,센트럴파크2,미추홀구청입구,"숭의가든, 성진물텀벙, 8퍼센트커피 논현점, 진흥각, 두툼당"
9,중국성,버스,GGB162000136,북구도서관,"참새초밥 부평점, 진시황감자탕, 타니스 청라레스토랑, 미들웨이, 정밀",지하철,부평구청역,계양역,"해가연, 중화루, 분아뜰리에, 대성불고기, 배터지는게장"


In [27]:
result_df["단계"]

0        신승반점
1         만다복
2         십리향
3          청관
4         공화춘
5        문차이나
6       서산밴댕이
7    아키라커피 본점
8          풍미
9         중국성
Name: 단계, dtype: object